In [ ]:
import pandas as pd

df = pd.read_csv('C:/Users/david/Documents/ENEL645Project/AirOnTimeCSV/airOT/files/airOT2000.csv')


In [4]:
df.shape

(5683047, 46)

In [5]:
mask = df.CANCELLED > 0.5
column_name = 'LABEL'
df.loc[mask, column_name] = 3

# create label column
# 3-CANCELLED
# 2-DIVERTED
# 1-ARR_DEL15
# 0-ON_TIME (to 15 mins late)
# -1-OTHER (shouldn't be any others)

# lfdf = cleandf.withColumn("LABEL", when(cleandf.CANCELLED > 0.5, 3)\
#                    .otherwise(when(cleandf.DIVERTED > 0.5, 2)\
#                              .otherwise(when(cleandf.ARR_DEL15 > 0.5, 1)\
#                                        .otherwise(when(cleandf.ARR_DEL15 < 0.5, 0)\
#                                                   .otherwise(-1)))))

In [6]:
mask = df.DIVERTED > 0.5
column_name = 'LABEL'
df.loc[mask, column_name] = 2

In [7]:
mask = df.ARR_DEL15 > 0.5
column_name = 'LABEL'
df.loc[mask, column_name] = 1

In [8]:
mask = df.ARR_DEL15 < 0.5
column_name = 'LABEL'
df.loc[mask, column_name] = 0

In [23]:
df2 = df[["YEAR", "MONTH", "DAY_OF_WEEK", "UNIQUE_CARRIER", "ORIGIN",
               "DEST", "DEST_STATE_ABR", "CRS_DEP_TIME", "DISTANCE", "LABEL"]]
df2 = df2.loc[df2['ORIGIN'] == 'IAH']

In [55]:
df_OT = df2.loc[df2['LABEL'] == 0]
df_DL = df2.loc[df2['LABEL'] == 1]

df_OT = df_OT.head(df_DL.shape[0])

df2 = pd.concat([df_OT, df_DL])

In [40]:
cancelled = 0
diverted = 0
delayed = 0
onTime = 0
for i in df2["LABEL"]:
    if (i == 0):
        onTime+=1
    elif (i == 1):
        delayed+=1
    elif (i == 2):
        diverted+=1
    elif (i == 3):
        cancelled+=1
print("On Time: ", onTime)
print("Delayed: ", delayed)
print("Diverted: ", diverted)
print("Cancelled: ", cancelled)

On Time:  26891
Delayed:  26891
Diverted:  0
Cancelled:  0


In [61]:
from sklearn.utils import shuffle
df2 = shuffle(df2)

X = df2.drop(columns=['LABEL', 'YEAR', 'MONTH', 'DAY_OF_WEEK'])
y = df2['LABEL']


In [62]:
# Uses OneHotEncoder
import category_encoders as ce

ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
X = ohe.fit_transform(X)
X.head()

,UNIQUE_CARRIER_CO,UNIQUE_CARRIER_NW,UNIQUE_CARRIER_WN,UNIQUE_CARRIER_US,UNIQUE_CARRIER_UA,UNIQUE_CARRIER_DL,UNIQUE_CARRIER_TW,UNIQUE_CARRIER_AA,UNIQUE_CARRIER_HP,ORIGIN_IAH,...,DEST_STATE_ABR_UT,DEST_STATE_ABR_NM,DEST_STATE_ABR_AL,DEST_STATE_ABR_KY,DEST_STATE_ABR_RI,DEST_STATE_ABR_WI,DEST_STATE_ABR_PR,DEST_STATE_ABR_AR,CRS_DEP_TIME,DISTANCE
399036,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2038,191.0
1119522,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1311,845.0
1210174,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1640,1034.0
5236056,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1853,1507.0
2015797,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2035,667.0


In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import time

NAME = 'IAH-2000-64x2-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

#create model
model = Sequential()

#get number of columns in training data
n_cols = X.shape[1]

#add model layers
model.add(Dense(16, activation='relu', input_shape=(n_cols,)))
model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2, callbacks=[tensorboard])

Train on 43025 samples, validate on 10757 samples
Epoch 1/10
43025/43025 [==============================] - 8s 182us/sample - loss: 0.8290 - acc: 0.5294 - val_loss: 0.8309 - val_acc: 0.5141
Epoch 2/10
43025/43025 [==============================] - 7s 173us/sample - loss: 0.7355 - acc: 0.5572 - val_loss: 0.7189 - val_acc: 0.5432
Epoch 3/10
43025/43025 [==============================] - 7s 172us/sample - loss: 0.7370 - acc: 0.5597 - val_loss: 0.7072 - val_acc: 0.5763
Epoch 4/10
43025/43025 [==============================] - 7s 173us/sample - loss: 0.7281 - acc: 0.5673 - val_loss: 0.7119 - val_acc: 0.5836
Epoch 5/10
43025/43025 [==============================] - 7s 171us/sample - loss: 0.7181 - acc: 0.5748 - val_loss: 0.7093 - val_acc: 0.5871
Epoch 6/10
43025/43025 [==============================] - 7s 171us/sample - loss: 0.7146 - acc: 0.5729 - val_loss: 0.6736 - val_acc: 0.5869
Epoch 7/10
43025/43025 [==============================] - 7s 172us/sample - loss: 0.6932 - acc: 0.5749 - val_l